In [55]:
pre=os.getcwd()+"/"
import os
os.system("unzip "+pre+"data_sushant.zip")

256

In [13]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [29]:
use_tpu=False

if(use_tpu and colab):
  import tensorflow as tf
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(resolver)
  # This is the TPU initialization code that has to be at the beginning.
  tf.tpu.experimental.initialize_tpu_system(resolver)
  print("All devices: ", tf.config.list_logical_devices('TPU'))
  strategy = tf.distribute.TPUStrategy(resolver)

In [30]:
sentiment_path=pre+"data_sushant/sentiment"
hate_path=pre+"data_sushant/hate"
irony_path=pre+"data_sushant/irony"

paths=[sentiment_path,hate_path,irony_path]

In [31]:
# loading the training dataframe
import pandas as pd

trains=[]
validation=[]
for path in paths:
    df = pd.read_csv(path+"/"+"train_text.txt", delimiter = "\n", names=["Data"])
    x_sentiment_data=df["Data"]
    print("="*20+"training data"+"="*20)
    print(x_sentiment_data.shape)
    
    df = pd.read_csv(path+"/"+"train_labels.txt", delimiter = "\n", names=["labels"])
    x_sentiment_labels=df["labels"]
    print(x_sentiment_labels.shape)
    
    trains.append([x_sentiment_data,x_sentiment_labels])
    print("="*20+"validation data"+"="*20)
    
    df = pd.read_csv(path+"/"+"val_text.txt", delimiter = "\n", names=["Data"])
    x_sentiment_data=df["Data"]
    print(x_sentiment_data.shape)
    
    df = pd.read_csv(path+"/"+"val_labels.txt", delimiter = "\n", names=["labels"])
    x_sentiment_labels=df["labels"]
    print(x_sentiment_labels.shape)
    
    validation.append([x_sentiment_data,x_sentiment_labels])


====================training data====================
(36373,)
(45615,)
====================validation data====================
(1623,)
(2000,)
====================training data====================
(8993,)
(9000,)
====================validation data====================
(989,)
(1000,)
====================training data====================
(2862,)
(2862,)
====================validation data====================
(955,)
(955,)


In [32]:
#build data set
import functools
def data_labels(lst):
    return pd.concat([lst[0],lst[1]],axis=1)

#training data
res=list(map(lambda x: data_labels(x),trains))
res=(functools.reduce(lambda a,b : pd.concat([a,b]),res))
train=res.dropna()

#testing data
res=list(map(lambda x: data_labels(x),validation))
res=(functools.reduce(lambda a,b : pd.concat([a,b]),res))
test=res.dropna()

In [33]:
print(train.shape,test.shape)

(48228, 2) (3567, 2)


In [34]:
print(train["labels"].value_counts())
print(test["labels"].value_counts())

1    21661
2    14259
0    12308
Name: labels, dtype: int64
1    1591
0    1324
2     652
Name: labels, dtype: int64


In [35]:
#transformer based models do not require preprocessing but in general it can help alot understanding about notions

import re

from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.corpus import stopwords 
import string

import nltk
nltk.download('stopwords')

stopwords_english = stopwords.words('english') 
#print(stopwords_english)


print('\nPunctuation\n')
print(string.punctuation)

def remove_links(sentance):
    sentance = re.sub(r"http\S+", "", sentance) #remove html links
    sentance = BeautifulSoup(sentance, 'lxml').get_text() #remove html tags
    return sentance

def remove_hastags_and_at(sentance):
    # remove hashtags
    
    # only removing the hash # sign from the word
    sentance = re.sub(r'#', '', sentance)
    
    blank=""
    for word in sentance.split(" "):
        if(word.startswith("@")):
            word=""
        blank+=word+" "
        
    return blank.strip()

def remove_special_symbols(sentance):
    #this also removes emoticons
    blank=[]
    for word in sentance.split(" "):
        word=re.sub('[^A-Za-z0-9]+', '', word)
        word = re.sub("\S*\d\S*", "", word) # remove words with numbers
        blank.append(word)
    return " ".join(blank)

def remove_stop_words(sentance):
    tweets_clean=[]
    for word in sentance.split(" "):
        if (word.lower() not in stopwords_english and  # remove stopwords
            word.lower() not in string.punctuation):  # remove punctuation
            tweets_clean.append(word.lower())
    return " ".join(tweets_clean)
   
def remove_retweet(sentance):
    # remove old style retweet text "RT"
    sentance = re.sub(r'^RT[\s]+', '', sentance)
    return sentance

def preprocessing(sentance):
    sentance=remove_links(sentance)
    sentance=remove_retweet(sentance)
    sentance=remove_hastags_and_at(sentance)
    sentance=remove_special_symbols(sentance)
    #sentance=remove_stop_words(sentance) #stop words are important hence not removing
    return sentance
    
train["Data"].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!

Punctuation

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


0    QT @user In the original draft of the 7th book...
1    Ben Smith / Smith (concussion) remains out of ...
2    Sorry bout the stream last night I crashed out...
3    Chase Headley's RBI double in the 8th inning o...
4    @user Alciato: Bee will invest 150 million in ...
Name: Data, dtype: object

In [36]:
#apply preprocessing
train["Data"]=train["Data"].apply(preprocessing)
test["Data"]=test["Data"].apply(preprocessing)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
print(train.shape)
print(test.shape)

(48228, 2)
(3567, 2)


In [38]:
#follow the tutorial for bert

In [39]:
!pip install ktrain

In [40]:
from ktrain import text
import ktrain

In [41]:
(x_train,y_train),(x_test,y_test),preprocess=text.texts_from_df(train,
                                             text_column="Data", label_columns=["labels"], 
                                             val_df=test,
                                             maxlen=400, val_pct=0.1, ngram_range=1,
                                             preprocess_mode='bert',
                                             random_state=2, verbose=1)

['labels_0', 'labels_1', 'labels_2']
   labels_0  labels_1  labels_2
0       0.0       0.0       1.0
1       0.0       1.0       0.0
2       0.0       1.0       0.0
3       0.0       1.0       0.0
4       0.0       0.0       1.0
['labels_0', 'labels_1', 'labels_2']
   labels_0  labels_1  labels_2
0       0.0       1.0       0.0
1       0.0       0.0       1.0
2       1.0       0.0       0.0
3       0.0       1.0       0.0
4       0.0       1.0       0.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [42]:
x_train[0].shape #see the same shape

(48228, 400)

In [43]:
model= text.text_classifier(name="bert",
                            train_data=(x_train,y_train),
                            preproc=preprocess)

Is Multi-Label? False
maxlen is 400
done.


In [52]:
learner=ktrain.get_learner(model=model,
                           train_data=(x_train,y_train),
                           val_data=(x_test,y_test),
                           batch_size=6)

In [45]:
#learner.lr_find(show_plot=True)

#optimal learning rate is 

In [53]:
learner.fit_onecycle(lr=2e-5,epochs=1)



begin training using onecycle policy with max lr of 2e-05...
  39/8038 [..............................] - ETA: 2:54:07 - loss: 1.1353 - accuracy: 0.4017

KeyboardInterrupt: ignored

In [ ]:
predictor=ktrain.get_predictor(learner.model,preprocess)


In [ ]:
data=["you are fucking cunt", "go fuck yourself"]

In [ ]:
predictor.predict(data)

In [ ]:
predictor.save(os.getcwd()+"/bert")

In [ ]:
predictor.load(os.getcwd()+"/bert")
data=["you are fucking cunt", "go fuck yourself"]
predictor.predict(data)